#AWS EC2 With Boto3  
EC2 = Elastic Compute Cloud

##Tasks:
1. Get EC2 Info
2. Create Security Group
3. Create EC2
4. Start/Stop/Reboot EC2
5. Monitor Instance

Import the boto3 module first

In [ ]:
import boto3

Task 1: Getting information about existing EC2 instances

In [ ]:
ec2 = boto3.client("ec2")
ec2.describe_instances()

Task 2: Create Security Group (and also the Key Pair)

In [ ]:
ec2 = boto3.client("ec2")
ec2.describe_instances()